In [1]:
import subprocess
import threading
import queue

def enqueue_output(out, queue):
    for line in iter(out.readline, b''):
        queue.put(line)
    out.close()

# Create a queue to hold the output lines
output_queue = queue.Queue()

# Start a subprocess (e.g., running the 'ping' command)
process = subprocess.Popen(['ping', 'google.com'], stdout=subprocess.PIPE, stderr=subprocess.PIPE, bufsize=1, universal_newlines=True)

# Start a thread to read the subprocess output
thread = threading.Thread(target=enqueue_output, args=(process.stdout, output_queue))
thread.daemon = True  # Thread dies with the program
thread.start()

# Read lines from the queue
while True:
    try:
        line = output_queue.get_nowait()  # Non-blocking read
    except queue.Empty:
        if process.poll() is not None:
            break
    else:
        print(line, end='')  # Print the line

# Wait for the thread to finish
thread.join()



Pinging google.com [142.250.193.78] with 32 bytes of data:
Reply from 142.250.193.78: bytes=32 time=4ms TTL=118
Reply from 142.250.193.78: bytes=32 time=4ms TTL=118
Reply from 142.250.193.78: bytes=32 time=4ms TTL=118
Reply from 142.250.193.78: bytes=32 time=4ms TTL=118

Ping statistics for 142.250.193.78:
    Packets: Sent = 4, Received = 4, Lost = 0 (0% loss),
Approximate round trip times in milli-seconds:
    Minimum = 4ms, Maximum = 4ms, Average = 4ms


In [1]:
import subprocess
import threading
import queue
import os

def enqueue_output(out, queue):
    for line in iter(out.readline, ''):
        queue.put(line)
    out.close()

def run_command(command, queue, pid_container):
    process = subprocess.Popen(command, shell=True, stdout=subprocess.PIPE,
                               stderr=subprocess.STDOUT, 
                               text=True,
                               preexec_fn=os.setsid
                               )
    
    pid_container[0] = process.pid 
    t = threading.Thread(target=enqueue_output,
                         args=(process.stdout, queue))
    t.daemon = True
    t.start()

# Example usage
output_queue = queue.Queue()
pid_container = [None]

run_command('ping google.com -c 4', output_queue, pid_container)

# Read lines from the queue
while True:
    try:
        line = output_queue.get_nowait()  # Non-blocking read
    except queue.Empty:
        if pid_container[0] is None:
            break
    else:
        print(line, end='')  # Print the line

# Wait for the thread to finish
threading.Event().wait(1)  # Small delay to ensure all output is processed


AttributeError: module 'os' has no attribute 'setsid'

In [2]:
import subprocess
import threading
import queue
import os
import platform

def enqueue_output(out, queue):
    for line in iter(out.readline, ''):
        queue.put(line)
    out.close()

def run_command(command, queue, pid_container):
    if platform.system() == 'Windows':
        process = subprocess.Popen(command, shell=True, stdout=subprocess.PIPE,
                                   stderr=subprocess.STDOUT, 
                                   text=True)
    else:
        process = subprocess.Popen(command, shell=True, stdout=subprocess.PIPE,
                                   stderr=subprocess.STDOUT, 
                                   text=True,
                                   preexec_fn=os.setsid)
    
    pid_container[0] = process.pid 
    t = threading.Thread(target=enqueue_output,
                         args=(process.stdout, queue))
    t.daemon = True
    t.start()

# Example usage
output_queue = queue.Queue()
pid_container = [None]

run_command('ping google.com -c 4', output_queue, pid_container)

# Read lines from the queue
while True:
    try:
        line = output_queue.get_nowait()  # Non-blocking read
    except queue.Empty:
        if pid_container[0] is None:
            break
    else:
        print(line, end='')  # Print the line

# Wait for the thread to finish
threading.Event().wait(1)  # Small delay to ensure all output is processed


Access denied. Option -c requires administrative privileges.


KeyboardInterrupt: 

In [1]:
import subprocess
import threading
import queue
import os
import time
import signal
import platform

def enqueue_output(out, queue):
    for line in iter(out.readline, ''):
        queue.put(line)
    out.close()

def run_command(command, queue, pid_container):
    if platform.system() == 'Windows':
        process = subprocess.Popen(command, shell=True, stdout=subprocess.PIPE,
                                   stderr=subprocess.STDOUT, 
                                   text=True)
    else:
        process = subprocess.Popen(command, shell=True, stdout=subprocess.PIPE,
                                   stderr=subprocess.STDOUT, 
                                   text=True,
                                   preexec_fn=os.setsid)
    
    pid_container[0] = process.pid 
    t = threading.Thread(target=enqueue_output,
                         args=(process.stdout, queue))
    t.daemon = True
    t.start()

def execute_command(command, timeout=5):
    q = queue.Queue()
    pid_container = [None]
    run_command(command, q, pid_container)
    output = ''
    start_time = time.time()

    while True:
        elapsed_time = time.time() - start_time
        if elapsed_time > timeout:
            if platform.system() == 'Windows':
                os.kill(pid_container[0], signal.SIGTERM)
            else:
                os.killpg(pid_container[0], signal.SIGKILL)
            break

        try:
            line = q.get_nowait()
        except queue.Empty:
            line = None 

        if line:
            output += line 

    return output[-2000:]

# Example usage
command_output = execute_command('ping google.com -c 4', timeout=5)
print(command_output)


OSError: [WinError 87] The parameter is incorrect

In [2]:
import os

def build_directory_tree(path, prefix="", ignore=None, is_last=False):
    """Build the directory tree in tree-like format.
    
    Args:
     - path: the starting directory path.
     - prefix: Prefix for the current item, used for recursion.
     - ignore: List of directory names to ignore.
     - is_last: Flag to indicate if the current item is the last in its parent directory.

     Returns:
      - A string representation of the directory tree.
    """
    if ignore is None:
        ignore = []

    if os.path.basename(path) in ignore:
        return ""
    
    output = ""
    indent = '|    ' if not is_last else '    '

    if os.path.isdir(path):
        # It's a directory, add its name to the output and then recurse into it
        output += prefix + "|-- " + os.path.basename(path) + "/\n"

        # List items in the directory
        items = os.listdir(path)
        for index, item in enumerate(items):
            item_path = os.path.join(path, item)
            output += build_directory_tree(item_path,
                                           prefix=prefix + indent, 
                                           ignore=ignore,
                                           is_last=index == len(items) - 1)
    else:
        # It's a file, add its name to the output
        output += prefix + "|-- " + os.path.basename(path) + "\n"

    return output

# Example usage
directory_path = "/path/to/your/directory"
print(build_directory_tree(directory_path, ignore=[".git", "__pycache__"]))


|-- directory

